https://www.kaggle.com/xhlulu/siim-covid-19-convert-to-jpg-256px

In [1]:
import pandas as pd
import numpy as np
import pydicom
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

import os

In [2]:
df = pd.read_csv('/workspace/data/df_train_study_level.csv')

In [3]:
save_path = "/workspace/data/train_640_2/"
if os.path.exists(save_path) == False:
    os.makedirs(save_path)

In [4]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def save_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    png_path = save_path + path.split('/')[-1].replace("dcm", "png")
    cv2.imwrite(png_path, data)

In [5]:
dicom_path_list = df.dicom_path

In [7]:
from multiprocessing import Pool
import time

pool = Pool(processes=10)

with tqdm(total=len(dicom_path_list)) as t:
    for _ in pool.imap_unordered(save_xray, dicom_path_list):
        t.update(1)

  3%|▎         | 161/6334 [00:05<03:17, 31.21it/s]/opt/conda/lib/python3.8/site-packages/pydicom/pixel_data_handlers/numpy_handler.py:341: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)
100%|██████████| 6334/6334 [04:35<00:00, 23.00it/s]


In [10]:
df["npy_path"] = [save_path + path.split('/')[-1].replace("dcm", "png") for path in df["dicom_path"]]

In [11]:
df.to_csv('/workspace/data/df_train_study_level_npy640_2.csv')